In [1]:
import pandas as pd
import numpy as np
import os
import sys
import datetime 
from functools import partial
import math
from itertools import count

In [2]:
folder = os.getcwd() + '/datasets/'
files = os.listdir(folder)

for file_name in files.copy():
    if '.txt' in file_name:
        files.remove(file_name)

In [3]:
dfs : dict[pd.DataFrame] = {}

for file_name in files:
    dfs[file_name] = pd.read_csv(folder+file_name)

In [4]:
def convert_dates(text:str):
    return datetime.datetime.strptime(text,'%Y-%m-%dT%H:%M:%S.%fZ')

In [5]:
years = []
for key,value in dfs.items():
    date_col_name = value.columns[0]
    try:
        value[date_col_name] = value[date_col_name].map(convert_dates)
    except TypeError:
        pass

In [6]:
for values in dfs.values():
    _,*col_names = list(values.columns)
    col_names.insert(0,'day')
    values.columns = col_names

In [7]:
for key,values in dfs.copy().items():
    dfs[key]['day'] = values['day'].map(str)
    dfs[key]['day'] = values['day'].map(lambda x : x.split(' ')[0])
    dfs[key]['day'] = values['day'].map(lambda x : ''.join(x.split('-')[:-1]))

In [18]:
dfs_means = []
for key,values in dfs.copy().items():
    unique_months = values['day'].unique()
    
    df_unique_months = {}
    for month in unique_months:
        unique_month_values = values[values['day'] == month]
        unique_month_values = unique_month_values.drop('day',axis=1)
        
        for mean_value,index in zip(unique_month_values.mean(),unique_month_values.mean().index):
            if df_unique_months.get(index,False):
                df_unique_months[index].append(mean_value)
            else:
                df_unique_months[index] = [mean_value]
        if df_unique_months.get('day',False):
            df_unique_months['day'].append(month)
        else:
            df_unique_months['day'] = [month]
        df_unique_months_df = pd.DataFrame(data=df_unique_months)
        dfs_means.append(df_unique_months_df)
    

In [21]:
print(len(dfs_means))

2306


In [30]:
df_final = pd.concat(dfs_means,ignore_index=True)


In [33]:
df_final.to_csv('./updated_datasets_distrubutions/nan_ds.csv',index=False)